# Marchantia Image Processing (Updated January 6th 2020)

In [ ]:
# MODULE IMPORT
import os # For manipulating system information (file & folder paths and related)
from skimage import io # For importing and saving images
from skimage import exposure # For improving contrast and brightness in an image
from skimage import img_as_float # To transform an 8 bit image to floating points
from skimage.color import * # For color manipulation and transforming
import matplotlib.pyplot as plt # for plotting all the figures
from skimage import filters # for thresholding algorithms
from skimage import morphology # for dilation, erosion and other binary transforms
import numpy as np # for array manipulation
from IPython.display import clear_output # Used to avoid output cluttering
from tkinter import Tcl # Module used for intelligent sorting of lists
import scipy.ndimage as ndi 
%matplotlib inline
# Required for viewing images in silico

# The modules listed below may or may not be necessary depending on the code executed

#import re # Regex module for powerful manipulation of strings
# import skimage.feature as skfeat # for detecting ROIs and OOIs
# from math import *
# from scipy import stats # Module used for statistical analysis of data obtained from images

In [ ]:
def getFiles(days, f_folder_name='FLUO', bf_folder_name='BF', oper_sys='MAC'):
    
    '''
    Function used to obtain all filepaths for all the images taken. Prerequisetes are one main folder which contains
    subfolders. Each of these subfolders is named accordingly to the plant type analyzed (WT, BFP, VENUS, etc).
    Within the subfolders there can be two options:
    
        1. The folder corresponds to the Wildtype plants, in which case there won't be any more subfolders and
        all the images to be analyzed are within this folder.
        
        2. The folder corresponds to a plant which has a fluorophore. In this case there will be two additional
        subfolders, one which will contain the brightfield images and one which will contain the fluorescent images.
        
    Function parameters:
    
        days: Number of days the experiment was conducted (this could be theoretically extracted from filepath)
        f_folder_name: Name of the fluorescent folder, defaults to FLUO. This must be the SAME FOR ALL PLANT TYPES.
        bf_folder_name: Name of the brightfield folder, defaulst to BF. This must be the SAME FOR ALL PLANT TYPES.
        oper_sys: Type of operating system using while running the code, defaults to MAC. Valid parameters are only
        MAC and WINDOWS (with capital letters).
        
    Output:
    
        all_files: Dictionary with all the files found sorted into plant types and different days.
    '''
    
    if oper_sys == 'MAC':

        search_directory = str(input('Please enter the directory where the image folders and files are: '))
        # Error handling when entering invalid directory paths
        try:
            os.chdir(search_directory)
        except:
            print('This is not a valid directory, please enter a new directory.')
            return None

        paths = [os.path.join(search_directory, f) for f in os.listdir(search_directory) if f != '.DS_Store']
        paths.sort()

        all_files = {}

        for j in range(len(paths)):
                all_files['{}'.format(paths[j].rsplit('/', 1)[1])] = {}

        for h in range(len(all_files)):
            if paths[h].rsplit('/',1)[1].upper() == 'WT' or paths[h].rsplit('/',1)[1].upper() == 'WILDTYPE':
                all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF'] = {}
                for i in range(days):
                    all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF']['{}'.format('DAY_{}'.format(i))] = []
            elif paths[h].rsplit('/',1)[1] != 'WT' and paths[h].rsplit('/',1)[1] != '.DS_Store':
                all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['FLUO'] = {}
                all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF'] = {}
                for i in range(days):
                    all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['FLUO']['{}'.format('DAY_{}'.format(i))] = []
                    all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF']['{}'.format('DAY_{}'.format(i))] = []

        for path in paths:

            if path.rsplit('/',1)[1].upper() == 'WT' or path.rsplit('/',1)[1].upper() == 'WILDTYPE':

                print('This folder contains wild type Mp photos')
                os.chdir(path)
                files = filter(os.path.isfile, os.listdir(path))
                files = [os.path.join(path, f) for f in files if f != '.DS_Store']
                sorted_files = Tcl().call('lsort', '-dict', files)

                for file in sorted_files:

                    filename, ext = os.path.splitext(file)
                    day = filename.rsplit(' ', 2)[1] # This gives us the specific day
                    plant_type = filename.rsplit(' ', 2)[0].rsplit('/', 1)[1] # This gives us the plant type

                    for key in list(all_files[path.rsplit('/',1)[1].upper()]['BF'].keys()):

                        if day == key.rsplit('_')[1]:

                            all_files[path.rsplit('/',1)[1].upper()]['BF'][key].append(file)

            else:

                print('This folder contains Brightfield and Fluorescent photos')
                os.chdir(path)
                folders = filter(os.path.isdir, os.listdir(path))
                folders = [os.path.join(path, f) for f in folders if f != '.DS_Store']
                '''
                We assume there will only be two folders, one of which will contain the fluorescent photos and
                one which will contain the brightfield photos

                IMPORTANT: Rename the folders to BF and FLUO for correct analysis
                '''
                for folder in folders:

                    if folder.rsplit('/', 1)[1] == bf_folder_name:

                        # Brightfield photographs
                        print('This folder contains brightfield photos')
                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files if f != '.DS_Store']
                        sorted_files = Tcl().call('lsort', '-dict', files)

                        for file in sorted_files:

                            filename, ext = os.path.splitext(file)
                            day = filename.rsplit(' ', 2)[1] # This gives us the specific day
                            plant_type = filename.rsplit(' ', 2)[0].rsplit('/', 1)[1] # This gives us the plant type

                            for key in list(all_files[path.rsplit('/',1)[1].upper()]['BF'].keys()):

                                if day == key.rsplit('_')[1]:

                                    all_files[path.rsplit('/',1)[1].upper()]['BF'][key].append(file)

                    elif folder.rsplit('/', 1)[1] == f_folder_name:

                        # Fluorescent photographs
                        print('This folder contains fluorescent photos')
                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files if f != '.DS_Store']
                        sorted_files = Tcl().call('lsort', '-dict', files)

                        for file in sorted_files:

                            filename, ext = os.path.splitext(file)
                            day = filename.rsplit(' ', 2)[1] # This gives us the specific day
                            plant_type = filename.rsplit(' ', 2)[0].rsplit('/', 1)[1] # This gives us the plant type

                            for key in list(all_files[path.rsplit('/',1)[1].upper()]['FLUO'].keys()):

                                if day == key.rsplit('_')[1]:

                                    all_files[path.rsplit('/',1)[1].upper()]['FLUO'][key].append(file)

        return all_files
                
    elif oper_sys == 'WINDOWS':
    
        search_directory = str(input('Please enter the directory where the image folders and files are: '))
        # Error handling when entering invalid directory paths
        try:
            os.chdir(search_directory)
        except:
            print('This is not a valid directory, please enter a new directory.')
            return None

        paths = [os.path.join(search_directory, f) for f in os.listdir(search_directory) if f != '.DS_Store']
        paths.sort()
        paths = list(map(lambda x: x.replace("\\", "/"), paths))

        all_files = {}

        for j in range(len(paths)):
                all_files['{}'.format(paths[j].rsplit('/', 1)[1])] = {}

        for h in range(len(all_files)):
            if paths[h].rsplit('/',1)[1].upper() == 'WT' or paths[h].rsplit('/',1)[1].upper() == 'WILDTYPE':
                all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF'] = {}
                for i in range(days):
                    all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF']['{}'.format('DAY_{}'.format(i))] = []
            elif paths[h].rsplit('/',1)[1] != 'WT' and paths[h].rsplit('/',1)[1] != '.DS_Store':
                all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['FLUO'] = {}
                all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF'] = {}
                for i in range(days):
                    all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['FLUO']['{}'.format('DAY_{}'.format(i))] = []
                    all_files['{}'.format(paths[h].rsplit('/', 1)[1])]['BF']['{}'.format('DAY_{}'.format(i))] = []

        for path in paths:

            if path.rsplit('/',1)[1].upper() == 'WT' or path.rsplit('/',1)[1].upper() == 'WILDTYPE':

                print('This folder contains wild type Mp photos')
                path = path.replace('/', '\\')
                os.chdir(path)
                files = filter(os.path.isfile, os.listdir(path))
                files = [os.path.join(path, f) for f in files if f != '.DS_Store']
                sorted_files = Tcl().call('lsort', '-dict', files)
                sorted_files = list(map(lambda x: x.replace("/", "\\"), sorted_files)) # Aqui tambien

                for file in sorted_files:

                    filename, ext = os.path.splitext(file)
                    day = filename.rsplit(' ', 2)[1] # This gives us the specific day
                    plant_type = filename.rsplit(' ', 2)[0].rsplit('\\', 1)[1] # This gives us the plant type

                    for key in list(all_files[path.rsplit('/',1)[1].upper()]['BF'].keys()):

                        if day == key.rsplit('_')[1]:

                            all_files[path.rsplit('\\',1)[1].upper()]['BF'][key].append(file)

            else:

                print('This folder contains Brightfield and Fluorescent photos')
                path = path.replace('/', '\\') # For windows
                os.chdir(path)
                folders = filter(os.path.isdir, os.listdir(path))
                folders = [os.path.join(path, f) for f in folders if f != '.DS_Store']
                folders = list(map(lambda x: x.replace('/', '\\'), folders))
                '''
                We assume there will only be two folders, one of which will contain the fluorescent photos and
                one which will contain the brightfield photos

                IMPORTANT: Rename the folders to BF and FLUO for correct analysis
                '''
                for folder in folders:

                    if folder.rsplit('\\', 1)[1] == bf_folder_name:

                        # Brightfield photographs
                        print('This folder contains brightfield photos')
                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files if f != '.DS_Store']
                        sorted_files = Tcl().call('lsort', '-dict', files)
                        sorted_files = list(map(lambda x: x.replace("/", "\\"), sorted_files))

                        for file in sorted_files:

                            filename, ext = os.path.splitext(file)
                            day = filename.rsplit(' ', 2)[1] # This gives us the specific day
                            plant_type = filename.rsplit(' ', 2)[0].rsplit('\\', 1)[1] # This gives us the plant type

                            for key in list(all_files[path.rsplit('\\',1)[1].upper()]['BF'].keys()):

                                if day == key.rsplit('_')[1]:

                                    all_files[path.rsplit('\\',1)[1].upper()]['BF'][key].append(file)

                    elif folder.rsplit('\\', 1)[1] == f_folder_name:

                        # Fluorescent photographs
                        print('This folder contains fluorescent photos')
                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files if f != '.DS_Store']
                        sorted_files = Tcl().call('lsort', '-dict', files)
                        sorted_files = list(map(lambda x: x.replace("/", "\\"), sorted_files))

                        for file in sorted_files:

                            filename, ext = os.path.splitext(file)
                            day = filename.rsplit(' ', 2)[1] # This gives us the specific day
                            plant_type = filename.rsplit(' ', 2)[0].rsplit('\\', 1)[1] # This gives us the plant type

                            for key in list(all_files[path.rsplit('\\',1)[1].upper()]['FLUO'].keys()):

                                if day == key.rsplit('_')[1]:

                                    all_files[path.rsplit('\\',1)[1].upper()]['FLUO'][key].append(file)

        for key in list(all_files.keys()):
           # Each of these keys is a plant types
            for subkey in list(all_files['{}'.format(key)].keys()):
                # Each of these subkeys is either WT of FLUO
                for subkey2 in list(all_files['{}'.format(key)]['{}'.format(subkey)].keys()):
                   # Each of these is a day of the experiment
                   all_files['{}'.format(key)]['{}'.format(subkey)]['{}'.format(subkey2)] = list(map(lambda x: x.replace("\\", "/"), all_files['{}'.format(key)]['{}'.format(subkey)]['{}'.format(subkey2)]))

        return all_files

In [ ]:
def resultDics(days, dictionary):
    
    '''
    Function used for the creation of dictionaries to store all the data obtained from the image analysis
    Input has to be the dictionary created with all the files
    '''
    
    all_results = {}
    
    for key in list(dictionary.keys()): 
        all_results['{}'.format(key)] = {}
        for day in range(days):
            all_results['{}'.format(key)]['DAY_{}'.format(day)] = {}
        for key2 in list(all_results['{}'.format(key)]):
            if key.upper() == 'WT' or key.upper() == 'WILDTYPE' or key.upper() == 'WILD TYPE':
                all_results['{}'.format(key)]['{}'.format(key2)]['AREA'] = []
            else:
                all_results['{}'.format(key)]['{}'.format(key2)]['AREA'] = []
                all_results['{}'.format(key)]['{}'.format(key2)]['INTENSITY'] = []
                
    return all_results

In [ ]:
# We now define all the functions we will use for the analysis

def openImage(file):
    
    '''
    Open the TIF or TIFF file as a three dimensional array for further manipulation
    Input:
        - file: string which corresponds to the filepath of the image
    Output:
        - img: three dimensional array that contains all the image information
    '''
    
    img = io.imread(file)
    
    return img

def plotImage(image, index, day, plant):

    '''
    Plot a specific image, given a specific plant number, day and plant type
    Input:
        - image: (Array) Three dimensional array containing the RGB image information.
        - index: (Int) Integer number, has to be a number between the plant number 1 and the plant number N, where N is the number of total plants of a specific type.
        - day: (Int) Integer number, has to be a number between Day 0 and Day N, where N is the total number of days the experiment ran.
        - plant: (Str) String corresponding to the plant type, for example: BFPN7, BFPLTI, VENUSN7, etc.
    Output:
        - Matplotlib figure showing the image requested
    '''
    
    specific_names = [name for name in all_names[day] if plant in name or plant.upper() in name or plant.lower() in name]
    
    plt.figure(figsize=(5,5), dpi=200)
    plt.title(specific_names[index])
    plt.imshow(image, cmap='gray')

def exporsureAndContrast(image):

    '''
    Function used for Contrast Limited Adaptive Image Equalization (CLAHE).
    Inputs:
        - image: Three dimensional (RGB) or One dimensional array (Greyscale) image information.
    Output:
        - clahe_img: Three dimensional (RGB) or One dimensional array (Greyscale) image, with an equalized histogram.
    '''
    
    clahe_img = exposure.equalize_adapthist(image, clip_limit=0.02)
    
    return clahe_img
    
def image2HSV(image):

    '''
    Function used for the conversion of RGB images to HSV color space (Hue Saturation Value)
    Input:
        - image: Three dimensional (RGB) array, containing image information. Array cannot be Greyscale or any other dimensions than RGB
    Output:
        - img_HSV: Three dimensional (HSV) array, containing image information in HSV color space
    '''
    
    img_HSV = rgb2hsv(image)
    
    return img_HSV

def firstSegmentation(image):

    '''
    Function used for segmentation of OOIs from a given image. It works
    by generating a threshold using the Otsu algorithm and the Value channel
    from the HSV image. Afterwards, it generates a binary image by applying
    the threshold value.
    Input:
        - image: Three dimensional HSV (optional RGB) image.
    Output:
        - binary: Binary thresholded image.
    '''

    thresh = filters.threshold_otsu(image[:,:,2])
    
    binary = image[:,:,2] < thresh-0.13
    
    return binary

def secondSegmentation(image):
    
    '''
    Function used for segmentation of OOIs from a given image. It works
    by generating a threshold using the Otsu algorithm and the Value channel
    from the HSV image. Afterwards, it generates a binary image by applying
    the threshold value.
    Input:
        - image: Three dimensional HSV (optional RGB) image.
    Output:
        - binary: Binary thresholded image.
    '''
    
    thresh = filters.threshold_otsu(image)
    
    binary = image > thresh
    
    return binary

def objectProcessing(image):
    
    '''
    Main function used for the removal of unwanted objects and enhacement of
    OOIs. It works by a series of morphological binary operations, consisting of
    binary closing, object removal, hole filling and erosion.
    Input:
        - image: Input image must be a binary image, RGB and Greyscale images will NOT work.
    Output:
        - img_eroded: Processed binary image, containing the segmented OOIs.
    '''
    
    img_closing = morphology.binary_closing(image, selem=np.ones((30,30)))
    
    img_wo = morphology.remove_small_objects(img_closing, min_size=10000)
    
    img_closing2 = morphology.binary_closing(img_wo, selem=np.ones((15,15)))
    
    img_fill = ndi.binary_fill_holes(img_closing2)
    
    img_eroded = morphology.binary_erosion(img_fill, selem=np.ones((5,5)))
    
    return img_eroded

def area_count(binary_img):
    
    '''
    Function used for calculating the area of a segmented object in a binary image. Methods just loops all pixels
    and counts a pixel if True (1).
    Input:
        - binary_img: Binary image that will be used for counting. Image MUST be binary, as the code is written
        as to loop over a 1D image.
    Output:
        - count: Number of pixels in the image that were True (1).
    '''
    
    count = 0
    
    for row in binary_img:
        
        for pixel in row:
        
            if pixel == False:

                continue

            else:

                count += 1
            
    return (count)

def areaAnalysis(file, objective=25):
    
    '''
    Function used for analyzing brightfield photos
    '''
    # Read the image
    img = openImage(file)
    # Convert to HSV color space for better segmentation
    HSV = image2HSV(img)
    # Segmentation
    segmented = firstSegmentation(HSV)
    # Object Processing
    wo = objectProcessing(segmented)
    # Calculate the area
    area = area_count(wo)*objective # Defaults to 4x objective which is 25

    return area

def getChannel(file):
    
    '''
    Function used for selecting the appropiate fluorescent photo analysis function, depending on the channel
    where the signal is strongest
    '''
    
    image = openImage(file)
    channels = []
    for i in range(len(image[0][0])):
        channels.append(np.sum(image[:, :, i]))
    channel = channels.index(max(channels))
    
    return channel

def redFluoAnalysis(file):

    '''
    Code to analyze fluorescent RGB images; used when the fluorophore used is detected on RED channel
    '''
    img = openImage(file)
    # We will use only the red channel for the fluorescence information 
    red_channel = img[:,:,0]
    # Segmentation
    segmented = secondSegmentation(red_channel)
    # We perform object processing within the Red Channel
    wo = objectProcessing(segmented)
    # We apply the mask obtained to each of the channels (R G and B)
    r = img[:,:,0]*wo
    g = img[:,:,1]*wo
    b = img[:,:,2]*wo
    # Final segmented image stacked into RGB
    final_rgb = np.dstack((r,g,b))
    # For the intensity calculation we convert the RGB image to greyscale
    greyscale_final = rgb2gray(final_rgb)
    # We calculate the intensity by calculating the sum of the greyscale values of the image
    intensity = round(np.sum(greyscale_final), 2)
    
    return intensity

def greenFluoAnalysis(file):
    
    '''
    Code to analyze fluorescent RGB images; used when the fluorophore used is detected on GREEN channel
    '''
    
    img = openImage(file)
    # We will use only the green channel for the fluorescence information 
    green_channel = img[:,:,1]
    # Segmentation
    segmented = secondSegmentation(green_channel)
    # We perform object processing within the Green Channel
    wo = objectProcessing(segmented)
    # We apply the mask obtained to each of the channels (R G and B)
    r = img[:,:,0]*wo
    g = img[:,:,1]*wo
    b = img[:,:,2]*wo
    # Final segmented image stacked into RGB
    final_rgb = np.dstack((r,g,b))
    # For the intensity calculation we convert the RGB image to greyscale
    greyscale_final = rgb2gray(final_rgb)
    # We calculate the intensity by calculating the sum of the greyscale values of the image
    intensity = round(np.sum(greyscale_final), 2)

    return intensity

def blueFluoAnalysis(file):
    
    '''
    Code to analyze fluorescent RGB images; used when the fluorophore used is detected on BLUE channel
    '''
    
    img = openImage(file)
    # We will use only the blue channel for the fluorescence information 
    blue_channel = np.multiply(img[:,:,2], 1.75)
    # Segmentation
    segmented = secondSegmentation(blue_channel)
    # We perform object processing within the Blue Channel
    wo = objectProcessing(segmented)
    # We apply the mask obtained to each of the channels (R G and B)
    r = img[:,:,0]*wo
    g = img[:,:,1]*wo
    b = img[:,:,2]*wo
    # Final segmented image stacked into RGB
    final_rgb = np.dstack((r,g,b))
    # For the intensity calculation we convert the RGB image to greyscale
    greyscale_final = rgb2gray(final_rgb)
    # We calculate the intensity by calculating the sum of the greyscale values of the image
    intensity = round(np.sum(greyscale_final), 2)

    return intensity

def analyzeAll(file_dictionary, result_dictionary):
    
    '''
    Function which will analyze all the image files found, according to their photo type.
    '''

    for plant_type in list(file_dictionary.keys()):
        # Loop through every plant type found in main folder
        photo_types = list(file_dictionary[plant_type].keys())

        for photo_type in photo_types:

            if photo_type == 'BF':
                print('Analyzing Brightfield Photos of:', plant_type)
                for day in list(file_dictionary[plant_type][photo_type].keys()):
                    for file in file_dictionary[plant_type][photo_type][day]:
                        '''
                        No need for obtaining channel since these are brightfield photos
                        '''
                        print('Calculating area of image {} of plant type'.format(file.rsplit('(', 1)[1].rsplit(')', 1)[0]), plant_type, 'of day', day.rsplit('_', 1)[1])
                        area = areaAnalysis(file, 25)
                        result_dictionary[plant_type][day]['AREA'].append(area)

            elif photo_type == 'FLUO':
                print('Analyzing Fluorescent Photos of:', plant_type)
                for day in list(file_dictionary[plant_type][photo_type].keys()):
                    '''
                    We have to get the channel index (R, G or B) to use the corresponding function to
                    analyze the fluorescent photos
                    '''
                    for file in file_dictionary[plant_type][photo_type][day]:
                        channel = getChannel(file)
                        if channel == 0:
                            print('Calculating intensity of image {} of plant type'.format(file.rsplit('(', 1)[1].rsplit(')', 1)[0]), plant_type, 'of day', day.rsplit('_', 1)[1])
                            intensity = redFluoAnalysis(file)
                            result_dictionary[plant_type][day]['INTENSITY'].append(intensity)
                        elif channel == 1:
                            print('Calculating intensity of image {} of plant type'.format(file.rsplit('(', 1)[1].rsplit(')', 1)[0]), plant_type, 'of day', day.rsplit('_', 1)[1])
                            intensity = greenFluoAnalysis(file)
                            result_dictionary[plant_type][day]['INTENSITY'].append(intensity)
                        elif channel == 2:
                            print('Calculating intensity of image {} of plant type'.format(file.rsplit('(', 1)[1].rsplit(')', 1)[0]), plant_type, 'of day', day.rsplit('_', 1)[1])
                            intensity = blueFluoAnalysis(file)
                            result_dictionary[plant_type][day]['INTENSITY'].append(intensity)
    
    return None

# Please test this code using the cells below, only run the analyzeAll function if you are certain all files and the result dictionaries are correct!

In [ ]:
testing_files = getFiles(5, 'Fluo', 'Campo Claro') # Get all image files
testing_dics = resultDics(5, testing_files) # Create all result dictionaries (will be used for storage of analysis results)
#analyzeAll(testing_files, testing_dics) # Analyze all files (from getFiles) and store the results in dictionary (from resultDics)

In [ ]:
#import pandas as pd
#from datetime import datetime

# Save the analyzed data to .csv file for export; save the file with date and time prefix to differentiate different analysis
#curr_datetime = datetime.now()
#curr_datetime_str = now.strftime("%d-%m-%Y_%H-%M")

#pd.DataFrame(all_areas).to_csv('/Users/Teoric/Desktop/{}_area'.format(curr_datetime_str), header=None, index=None)
#pd.DataFrame(all_intensities).to_csv('/Users/Teoric/Desktop/{}_intensities'.format(curr_datetime_str), header=None, index=None)

# FUNCTIONS BELOW NEED TO BE OPTIMIZED FOR CHANGES IN CODE! I.E: ALL_AREAS AND ALL_INTENSITIES ARRAYS ARE NO LONGER USED NEXT STEP IS TO UPDATE THESE FUNCTIONS

In [ ]:
def plotSpecific(means=False, stddev=False):
    
    '''
    Plot the results for the area or instensity of a specific type of plant
    Inputs are an array which has to correspond to one of the names noted above and
    a string which has to be the name of the array converted to a string.
    
    Also can plot the mean and standard error for each day is means and stderr is set to True
    
    As an example:
    
    array = [...]
    name = 'array'
    '''
    #os.chdir('/Users/Teoric/Desktop/5DAYS/')
    
    print('Please enter one of the following names (exactly as written) to view the data: ')
    
    area_keys = all_areas.keys()
    intensity_keys = all_intensities.keys()
    
    for key in area_keys:
        
        print('{}'.format(key))
        
    for key in intensity_keys:
        
        print('{}'.format(key))
        
    key_used = str(input('Enter type of plant to analyze: '))
    
    if key_used in area_keys:
        
        array = all_areas['{}'.format(key_used)]
        
    elif key_used in intensity_keys:
        
        array = all_intensities['{}'.format(key_used)]
        
    else:
        
        print('Key not valid, please enter a valid key.')
        return None
    
    plt.figure(figsize=(7,7), dpi=250)
    plt.title(key_used+' over 5 days')
    x_axis = [[0 for i in range(len(array[0]))],[1 for i in range(len(array[1]))],[2 for i in range(len(array[2]))],[3 for i in range(len(array[3]))],[4 for i in range(len(array[4]))]]
    color_list = ['mediumvioletred', 'crimson', 'slateblue', 'rebeccapurple', 'indigo']
    
    
    if means==True and stddev==False:
        
        # Contains the info for the mean of area or intensity for each day
        means_per_day = []
        
        for day in range(0, 5):
        
            plt.scatter(x_axis[day], array[day], s=2., alpha=0.3, marker='o', c=color_list[day])

            #Calculate means

            mean_day = np.mean(array[day])
            means_per_day.append(mean_day)
            
            plt.scatter(day, mean_day, s=15., marker='D', c=color_list[day])

        if key_used in intensity_keys:

            plt.xlabel('Day')
            plt.ylabel('Intensity (arbitrary units)')

        elif key_used in area_keys:

            plt.xlabel('Day')
            plt.ylabel('Area (\u03BCm\u00B2)')

        return
        
    elif means == True and stddev==True:
        
        # Contains the info for the mean of area or intensity for each day
        means_per_day = []
        stddev_per_day = []
        
        for day in range(0, 5):
        
            plt.scatter(x_axis[day], array[day], s=2., alpha=0.3, marker='o', c=color_list[day])

            # Calculate means

            mean_day = np.mean(array[day])
            means_per_day.append(mean_day)
            
            plt.scatter(day, mean_day, s=15., marker='D', c=color_list[day])
            
            # Calculate Standard Deviation
            
            stddev = stats.sem(array[day])
            stddev_per_day.append(stddev)
            
            plt.errorbar(day, mean_day, yerr=stddev*2, ecolor=color_list[day], elinewidth=1., barsabove=True, capsize=3.)
            

        if key_used in intensity_keys:

            plt.xlabel('Day')
            plt.ylabel('Intensity (arbitrary units)')

        elif key_used in area_keys:

            plt.xlabel('Day')
            plt.ylabel('Area (\u03BCm\u00B2)')

        return
        
    else:
        
        for day in range(0, 5):
        
            plt.scatter(x_axis[day], array[day], s=4., marker='o')

        if key_used in intensity_keys:

            plt.xlabel('Day')
            plt.ylabel('Intensity (arbitrary units)')

        elif key_used in area_keys:

            plt.xlabel('Day')
            plt.ylabel('Area (\u03BCm\u00B2)')

        return

In [ ]:
# Plot the graph using the function defined in the previous cell

plotSpecific(means=True, stddev=True)

In [ ]:
from labellines import labelLine, labelLines

def plotSpecificLINES(array, name):
    
    '''
    Plot the results for the area or instensity of a specific type of plant
    Inputs are an array which has to correspond to one of the names noted above and
    a string which has to be the name of the array converted to a string.
    
    As an example:
    
    array = [...]
    name = 'array'
    '''
    os.chdir('/Users/alejandro/Desktop/')
    
    plt.figure(figsize=(7,7), dpi=250)
    plt.title(name+' over 5 days')
    x_axis = [0, 1, 2, 3, 4] # Number of days
    
    plantdata = [[] for i in range(0, len(array[0]))]
    plant_legend = []    
    
    for day in range(0, 5):
        
        for plantnumber in range(0, len(array[day])):
        
            plantdata[plantnumber].append(array[day][plantnumber])
            plant_legend.append(str(plantnumber+1))
            
    for i in range(0, len(plantdata)):
        
        plt.plot(list(range(len(plantdata[i]))), plantdata[i], marker='o', markersize=2.5)
        
    plt.legend(plant_legend, loc='upper left', ncol=2)

    if 'intensity' in name:

        plt.xlabel('Day')
        plt.ylabel('Intensity (arbitrary units)')

    elif 'area' in name:

        plt.xlabel('Day')
        plt.ylabel('Area (square micrometers)')
        
    return